In [21]:
cd ~/data/modl_data/pipeline/contrast/prediction/136/

/storage/store/work/amensch/data/modl_data/pipeline/contrast/prediction/136


In [22]:
ls

_artifacts/  config.json  cout.txt  info.json  prediction.csv  run.json


In [23]:
from sklearn.externals.joblib import load
from sklearn.pipeline import Pipeline
import pandas as pd

from nilearn.input_data import MultiNiftiMasker
from nilearn.datasets import load_mni152_brain_mask
from modl.datasets.hcp import fetch_hcp

In [24]:
%matplotlib inline

def plot_confusion_matrix(conf_arr, labels):

    fig = plt.figure(figsize=(8, 8))
    plt.clf()
    ax = fig.add_subplot(111)
    ax.set_aspect(1)
    
    S = conf_arr.sum(axis=0)
    S[S == 0] = 1
    conf_arr = conf_arr / S
    
    res = ax.imshow(conf_arr,
                interpolation='nearest')

    width, height = conf_arr.shape

    plt.xticks(range(width))
    plt.yticks(range(height))
    ax.set_yticklabels(labels)
    ax.set_xticklabels(labels, rotation=90)
    fig.colorbar(res)
    
def plot_gram_matrix(G, labels):
    fig = plt.figure(figsize=(8, 8))
    plt.clf()
    ax = fig.add_subplot(111)
    ax.set_aspect(1)
    res = ax.imshow(G,
                interpolation='nearest')

    width, height = G.shape

    plt.xticks(range(width))
    plt.yticks(range(height))
    ax.set_yticklabels(labels)
    ax.set_xticklabels(labels, rotation=90)
    fig.colorbar(res)

In [28]:
cd _artifacts

/storage/store/work/amensch/data/modl_data/pipeline/contrast/prediction/136/_artifacts


In [29]:
load('estimator.pkl')

ValueError: Optimizer weight shape (30,) not compatible with provided weight shape (336, 200)

In [30]:
from modl.fixes import Model
label_encoder = load('label_encoder.pkl')
prediction = pd.read_csv('prediction.csv')

In [31]:
from keras.models import load_model

In [33]:
model = load_model('model_archi.keras')

ValueError: Optimizer weight shape (30,) not compatible with provided weight shape (336, 200)

In [37]:
import h5py 
f = h5py.File('model_archi.keras', 'r')

In [43]:
for weight in f['optimizer_weights']:
    print(weight)

adam_iter:0
adam_ms_0_1:0
adam_ms_1_1:0
adam_ms_2_1:0
adam_vs_0_1:0
adam_vs_1_1:0
adam_vs_2_1:0


In [ ]:
import h5py

In [ ]:
f = h5py.File('model_archi.keras', 'r+')
del f['optimizer_weights']

In [ ]:
f.close()

In [ ]:
model = load_model('model_archi.keras')

In [ ]:
for layer in model.layers:
    print(layer.name)

In [ ]:
f['optimizer']

In [ ]:
model.layers_by_depth[2][0].get_weights()

In [ ]:
estimator.named_steps['classifier'].models_['hcp'].layers_by_depth[1][0].get_weights()[0].shape

In [ ]:
model = estimator.named_steps['classifier']
coef = model.coef_

In [ ]:
res = np.linalg.svd(coef)

In [ ]:
res[1]

In [ ]:
model = estimator.steps[-1][1].stacked_model_

supervised_layers =  model.layers_by_depth[2]
embedding_layer =  model.layers_by_depth[3][0]

w0 = []
for layer in supervised_layers:
    this_w0, this_b0 = layer.get_weights()
    w0.append(this_w0)
w0 = np.concatenate(w0, axis=1)
w1 = embedding_layer.get_weights()[0]

coef = w1.dot(w0).T

print('Sparsity', (coef == 0).sum() / coef.size)

In [ ]:
prediction

In [ ]:
match = prediction['true_label'] == prediction['predicted_label']
prediction = prediction.assign(match=match)
prediction.sort_index(inplace=True)
datasets = prediction.index.get_level_values('dataset').unique().values

In [ ]:
labels_list = []
tasks_list = []
for dataset in datasets:
    first_subject = prediction.loc['test', dataset].index.get_level_values('subject')[0]
    data = prediction.loc[('test', dataset, first_subject)]
    labels = data.index.get_level_values('contrast').values
    tasks = data.index.get_level_values('task').values
    labels_list.append(labels)
    tasks_list.append(tasks)
labels = np.concatenate(labels_list)
tasks = np.concatenate(tasks_list)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
train_conf = confusion_matrix(prediction.loc['train', 'true_label'],
                        prediction.loc['train', 'predicted_label'], labels=labels)
test_conf = confusion_matrix(prediction.loc['test', 'true_label'],
                        prediction.loc['test', 'predicted_label'], labels=labels)

In [ ]:
plot_confusion_matrix(test_conf, labels)

In [ ]:
prediction['match'].groupby(level=['fold', 'dataset', 'task', 'contrast']).apply(np.mean)['test']

In [ ]:
idx = pd.IndexSlice
prediction.loc[idx['test']]

In [ ]:
prediction['match'].groupby(level=['fold', 'dataset']).apply(np.mean)

In [ ]:
classifier = estimator.named_steps['classifier']

In [ ]:
histories = classifier.histories_

In [ ]:
history.epoch

In [ ]:
for history, n_samples in zip(histories, classifier.n_samples_):
    plt.plot(np.array(history.epoch) * n_samples, history.history['val_acc'])

In [ ]:
titles = ["%s: %s" % (task, contrast) for task, contrast in zip(tasks, labels)]

In [ ]:
%matplotlib inline
from nilearn.plotting import plot_stat_map
from nilearn.image import index_img
from nilearn._utils import check_niimg

indices = label_encoder.transform(labels)

imgs = check_niimg(imgs)
imgs.get_data()

plt.close('all')

for i, title in zip(indices, titles):
    plot_stat_map(index_img(imgs, i), title=title, threshold=0)

In [ ]:
indices = label_encoder.transform(labels)
ncoef = coef[indices]
S = np.sqrt((ncoef ** 2).sum(axis=1))
ncoef = ncoef / S[:, np.newaxis]

In [ ]:
G = ncoef.dot(ncoef.T)
plot_gram_matrix(G, labels)

In [ ]:
np.mean(w0 == 0),np.mean(w1 == 0)

In [ ]:
np.histogram(w1)

In [ ]:
%matplotlib inline
n, bins, patches = plt.hist(w1.ravel(), bins=100, log=True)
plt.xscale('log')

In [ ]:
w1.ravel()[:300]

In [ ]:
(np.abs(coef[indices,:16]).sum())

In [ ]:
(np.abs(coef[indices,16:80])).sum()

In [ ]:
np.abs(coef[indices,80:80 + 256]).sum()

In [ ]:
np.mean(coef[indices, :16] == 0)

In [ ]:
np.mean(coef[indices, 16:80] == 0)

In [ ]:
np.mean(coef[indices, 80:256] == 0)